In [ ]:
import ee
shape = ee.FeatureCollection(" ") #feature collection
size = shape.size()
size1 = ee.Number(size).subtract(ee.Number(1))

total = ee.Image(" ") #downloaded annual image for ET
mws = ee.List.sequence(0,size1);
total = total.select('b1');

# Total pixels
pixelCount = total.reduceRegions(
    reducer = ee.Reducer.count(),
    collection = shape,
    scale = 1113.1949079327357
)
pixelCount = pixelCount.toList(size)

# Total Negative Pixels
negativePixels = total.lt(0)

negativePixelCount = negativePixels.reduceRegions(
    reducer = ee.Reducer.sum(),
    collection = shape,
    scale = 1113.1949079327357
)

def ll(k):
  pc = ee.Feature(pixelCount.get(k))
  id = pc.get('uid')
  nc = negativePixelCount.filter(ee.Filter.eq('uid', id)).first()
  p = ee.Number(pc.get('count'))
  nc = ee.Number(nc.get('sum'))
  val = p.subtract(nc)
  return pc.set('tot',val)

totalpix = ee.FeatureCollection(mws.map(ll))

total = total.expression(
            'ET>0?86400*ET:0', {
              'ET': total.select('b1')
        })

stats2 = total.reduceRegions(
    reducer = ee.Reducer.sum(),
    collection = shape,
    scale = 1113.1949079327357,
)

statsl = stats2.toList(size)

#year of interest
sd = start date

def res(m):
  f = ee.Feature(statsl.get(m))
  s = ee.Number(f.get('sum'))
  id = f.get('uid')
  feat = shape.filter(ee.Filter.eq('uid', id)).first()
  pix = totalpix.filter(ee.Filter.eq('uid', id)).first()
  pix = ee.Number(pix.get('tot'))
  val = s.divide(pix)
  return feat.set(sd, val)

shape = ee.FeatureCollection(mws.map(res));

task = ee.batch.Export.table.toAsset(**{
  'collection': shape,
  'description': 'ET_Annual',
  'assetId': ' ' #assetID
})
task.start()
